# Candidate User Generation Class

1. Instance 선언 : example = Candidate_user_generate()
2. List 반환 : example.get_person(user_id, cnt)
   - user_id : 현재 사용자의 id
   - cnt : 찾고자 하는 유사 사용자 수 (default = 3)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: ignored

In [ ]:
%cd '/content/drive/MyDrive/BKMS Team Project/Recommendation/User Recommend'

In [ ]:
!apt-get update
!apt-get install g++ openjdk-8-jdk python-dev python3-dev
# PostgreSQL 패키지
!pip install psycopg2-binary
# neo4j 패키지 설치
!pip install neo4j

In [ ]:
# import User_Recommend packages
import pandas as pd
import psycopg2

# neo4j 드라이버 import 및 함수 정의
from neo4j import GraphDatabase

## Content_Recommend packages
# import packages
import numpy as np

In [ ]:
class Neo4jConnection:
    
    def __init__(self, uri, user, pwd):
        self.__uri = uri
        self.__user = user
        self.__pwd = pwd
        self.__driver = None
        try:
            self.__driver = GraphDatabase.driver(self.__uri, auth=(self.__user, self.__pwd))
        except Exception as e:
            print("Failed to create the driver:", e)
        
    def close(self):
        if self.__driver is not None:
            self.__driver.close()
        
    def query(self, query, db=None):
        assert self.__driver is not None, "Driver not initialized!"
        session = None
        response = None
        try: 
            session = self.__driver.session(database=db) if db is not None else self.__driver.session() 
            response = list(session.run(query))
        except Exception as e:
            print("Query failed:", e)
        finally: 
            if session is not None:
                session.close()
        return response


class Candidate_user_generate:

  def __init__(self):
      # DB Connection Information
      self.connection_info = "host=147.47.200.145 dbname=teamdb5 user=team5 password=snugraduate port=34543" # PostgreSQL 연결 주소


  def connect_SQL(self): # DB에서 데이터를 불러옴
    # PostgreSQL 연결
    conn = psycopg2.connect(self.connection_info)

    try:
        # 테이블을 Pandas.Dataframe으로 추출 (user table, course table, enrollment table)
        self.user = pd.read_sql('SELECT id, username FROM auth_user ',conn)   # user table
        self.course = pd.read_sql('SELECT cid_int, cid, cname FROM check_course',conn)  # check_course table
        self.enroll = pd.read_sql('SELECT cid_int, user_id, cid from exp_enrollment',conn)  # enroll table  # exp -> rec
        
    except psycopg2.Error as e:
        # 데이터베이스 에러 처리
        print("DB error: ", e)
        
    finally:
        # 데이터베이스 연결 해제 필수!! 
        conn.close()

    self.enroll_idx = self.enroll.set_index('user_id')

  def connect_Neo4j(self, id, cnt): # 조건 쿼리, input : id - User ID / cnt : 몇명을 리턴할것인가? (default = 3)
    # DB Connection Information
    dbname = "teamdb5"
    uri_param = "bolt://147.47.200.145:37687"
    user_param = "team5"
    pwd_param = "snugraduate"
    apoc_string = 'CALL apoc.load.jdbc("jdbc:postgresql://localhost:34543/teamdb5?user=team5&password=snugraduate",'

    # Neo4j 연결
    conn = Neo4jConnection(uri=uri_param, user=user_param, pwd=pwd_param)
    # Cypher 쿼리 입력
    cypher = (apoc_string + '"SELECT cid_int, user_id,cid,up_id,gpa from rec_enrollment'
              'where up_id in (select max(re.up_id) from rec_enrollment re group by re.user_id)") YIELD row '
              'MERGE (sd:Student {id: row.user_id}) '                     # User Node 형성
              'MERGE (c:Course {cid_int: toInteger(row.cid_int)}) '  # 과목 Node 형성
              'MERGE (sd)-[e:Enrollment {gpa : row.gpa}]->(c) '  # 수강 Flow 형성
              'with sd,c '                                         
              'Match (sd{id:'+ str(id) + '})-->(c)<--(sd2) '         # User과 들은 과목을 들은 sd2를 찾아라
              'return distinct sd2.id, count(*) as cnt '              # sd2의 user id 반환
              'order by cnt desc '                                   # 겹치는 과목의 순서가 많은 사람부터
              'Limit '+ str(cnt)                                     # cnt명 반환
              )

    # Cypher 쿼리 실행 후 결과를 print
    response = conn.query(cypher, db=dbname)

    #모든 노드 삭제 - 초기화 필수!!
    conn.query('MATCH (n) DETACH DELETE n', db=dbname)    

    # 연결 종료 필수!
    conn.close()

    if response == None:      # 예외 처리 : 유사한 사용자가 없으면 전체 과목 리턴
      return list(self.course['cid']) # 전체 과목 id list
      
    person = pd.DataFrame([dict(record) for record in response])
    # print(person) # sd id 확인용
    if person.shape[0] < cnt :  # 예외 처리 : 특정 인원수가 안되면 그냥 전체 과목을 리턴
      return list(self.course['cid'])  # 전체 과목 id list

    # dictionray - 유저 확인용
    # cid_list = {}
    # for idx,i in person['sd2.id'].iteritems():
    #   cid_list[i] = list(self.enroll_idx['cid_int'].loc[i])
    # return cid_list

    cid_list = []
    for idx,i in person['sd2.id'].iteritems():
      cid_list+=list(self.enroll_idx['cid'].loc[i]) # 비슷한 과목을 들은 유저들의 과목을 리스트화
    cid_set = set(cid_list) # 중복 제거
    cid_list = list(cid_set) # 중복 제거

    for value in list(self.enroll_idx['cid'].loc[id]): # list에서 user가 들은 과목 제거
      cid_list.remove(value)

    # cid_list=[]
    # for i in cid_int_list:
    #   cid_list.append(self.course.set_index('cid_int')['cid'].loc[i])
    # cid_list

    cid_list.sort() # 오름차순 정렬

    return cid_list

  def get_person(self, id, cnt=3):  # 3명의 비슷한 유저를 찾아라
    self.connect_SQL()  # DB에서 데이터 로드
    return self.connect_Neo4j(id, cnt) # Candidate cnt명을 찾고 그들의 과목 cid_int 반환

In [ ]:
user_id = 3 # User id - input

In [ ]:
generator=Candidate_user_generate() # Class Instance 선언
# cid_list 생성
try:
  generated_list=generator.get_person(user_id) # input = user id , output = cid_int list
except psycopg2.Error as e:
  try:
    generated_list=generator.get_person(user_id) # input = user id  # 커넥팅 에러 방지차원으로 한번 더 실행
  except psycopg2.Error as e:
    try:
      generated_list=generator.get_person(user_id) # input = user id  # 커넥팅 에러 방지차원으로 한번 더 실행
    except psycopg2.Error as e:
      # 데이터베이스 에러 처리
      print("DB error: ", e)

In [ ]:
generated_list

['031.001',
 '031.002',
 '031.031',
 '031.032',
 '031.033',
 '032.001',
 '032.002',
 '032.003',
 '111.675',
 'M3239.004400']